In [2]:
import sys

from google.protobuf.json_format import MessageToJson
from client_wrapper import ServiceClient

import nlpserv_pb2 as nlp_messages
import nlpserv_pb2_grpc as nlp_service

In [3]:
from cabocha.analyzer import CaboChaAnalyzer
from rasa_nlu.tokenizers import Tokenizer, Token

def tokenize(text):         
    analyzer = CaboChaAnalyzer()
    tree = analyzer.parse(text)
    words=[]
    for chunk in tree:        
        for token in chunk:
            # print(token, token.pos)
            words.append(token.surface)

    running_offset = 0
    tokens = []
    for word in words:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        tokens.append(Token(word, word_offset))   
    return tokens

tokens=tokenize("お皿を二枚ください。")
for t in tokens:
    print(t.text, t.offset)

お 0
皿 1
を 2
二 3
枚 4
ください 5
。 9


In [4]:
text="お皿を二枚ください。"

analyzer = CaboChaAnalyzer()
tree = analyzer.parse(text)
msg_chunks=nlp_messages.NlCabochaChunks()
chunks=[]
for chunk in tree:
    msg_chunk=nlp_messages.NlCabochaChunk()
    msg_chunk.id=chunk.id
    if not chunk.additional_info is None:
        msg_chunk.additional_info=chunk.additional_info
    msg_chunk.feature_list.extend(chunk.feature_list)
    msg_chunk.func_pos=chunk.func_pos
    msg_chunk.head_pos=chunk.head_pos
    msg_chunk.link=chunk.link
    msg_chunk.score=chunk.score
    msg_chunk.token_pos=chunk.token_pos
    msg_chunk.next_link_id=chunk.next_link_id
    msg_chunk.prev_link_ids.extend(chunk.prev_link_ids)
    
    words=[]
    for token in chunk:
        # print(token, token.pos)
        word=nlp_messages.NlCabochaToken(surface=token.surface,
                                         id=token.id,
                                         additional_info=token.additional_info,
                                         feature_list=token.feature_list,
                                         ne=token.ne,
                                         normalized_surface=token.normalized_surface,
                                         pos=token.pos,
                                         pos1=token.pos1,
                                         pos2=token.pos2,
                                         pos3=token.pos3,
                                         ctype=token.ctype,
                                         cform=token.cform,
                                         genkei=token.genkei,
                                         yomi=token.yomi
                                        )
        words.append(word)
    msg_chunk.tokens.extend(words)
    chunks.append(msg_chunk)
    
msg_chunks.chunks.extend(chunks)
for chunk in msg_chunks.chunks:
    print(chunk.id, [word.surface for word in chunk.tokens])
    for word in chunk.tokens:
        print("\t", word.pos)

0 ['お', '皿', 'を']
	 接頭詞
	 名詞
	 助詞
1 ['二', '枚']
	 名詞
	 名詞
2 ['ください', '。']
	 動詞
	 記号


In [6]:
from rasa_nlu.tokenizers import Tokenizer, Token

def tokenize_msg(text, msg_chunks):         
    words=[]
    for chunk in msg_chunks.chunks:        
        for token in chunk.tokens:
            # print(token, token.pos)
            words.append(token.surface)

    running_offset = 0
    tokens = []
    for word in words:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        tokens.append(Token(word, word_offset))   
    return tokens

tokens=tokenize_msg("お皿を二枚ください。", msg_chunks)
for t in tokens:
    print(t.text, t.offset)

お 0
皿 1
を 2
二 3
枚 4
ください 5
。 9


In [21]:
from client_wrapper import ServiceClient
import nlpserv_pb2 as nlp_messages
import nlpserv_pb2_grpc as nlp_service

def run(text):
    serv = ServiceClient(nlp_service, 'CabochaNlpProcsStub', 'localhost', 50051)
    # Insert example metadata
    metadata = [('ip', '127.0.0.1')]
    response = serv.Tokenizer(
        nlp_messages.NlText(text=text),
        metadata=metadata
    )
    if response:
        print("response:")
        tokens=tokenize_msg(text, response)
        for t in tokens:
            print(t.text, t.offset)
        
        ## print pos
        for chunk in response.chunks:     
            print(chunk.id)            
            for token in chunk.tokens:
                pos_set=(token.pos, token.pos1, token.pos2, token.pos3)
                if "人名" in pos_set:
                    print("\t", token.id, token.surface, "person")
                elif "地域" in pos_set:
                    print("\t", token.id, token.surface, "location")
                else:
                    print("\t", token.surface, token.pos, token.pos1, 
                         token.pos2, token.pos3, "ctype", token.ctype,
                         [token.cform, token.genkei, token.yomi])

# pos=token.pos,
#    pos1=token.pos1,
#    pos2=token.pos2,
#    pos3=token.pos3,
#    ctype=token.ctype,
#    cform=token.cform,
#    genkei=token.genkei,
#    yomi=token.yomi
# run("お皿を二枚ください。")   
run("太郎は花子が読んでいる本を次郎に渡した")  
run("私は西海岸に住んでいます。")

response:
太郎 0
は 2
花子 3
が 5
読ん 6
で 8
いる 9
本 11
を 12
次 13
郎 14
に 15
渡し 16
た 18
0
	 0 太郎 person
	 は 助詞 係助詞 * * ctype * ['*', 'は', 'ハ']
1
	 2 花子 person
	 が 助詞 格助詞 一般 * ctype * ['*', 'が', 'ガ']
2
	 読ん 動詞 自立 * * ctype 五段・マ行 ['連用タ接続', '読む', 'ヨン']
	 で 助詞 接続助詞 * * ctype * ['*', 'で', 'デ']
	 いる 動詞 非自立 * * ctype 一段 ['基本形', 'いる', 'イル']
3
	 本 名詞 一般 * * ctype * ['*', '本', 'ホン']
	 を 助詞 格助詞 一般 * ctype * ['*', 'を', 'ヲ']
4
	 次 名詞 一般 * * ctype * ['*', '次', 'ツギ']
	 郎 名詞 一般 * * ctype * ['*', '郎', 'ロウ']
	 に 助詞 格助詞 一般 * ctype * ['*', 'に', 'ニ']
5
	 渡し 動詞 自立 * * ctype 五段・サ行 ['連用形', '渡す', 'ワタシ']
	 た 助動詞 * * * ctype 特殊・タ ['基本形', 'た', 'タ']
response:
私 0
は 1
西海岸 2
に 5
住ん 6
で 8
い 9
ます 10
。 12
0
	 私 名詞 代名詞 一般 * ctype * ['*', '私', 'ワタシ']
	 は 助詞 係助詞 * * ctype * ['*', 'は', 'ハ']
1
	 西海岸 名詞 一般 * * ctype * ['*', '西海岸', 'ニシカイガン']
	 に 助詞 格助詞 一般 * ctype * ['*', 'に', 'ニ']
2
	 住ん 動詞 自立 * * ctype 五段・マ行 ['連用タ接続', '住む', 'スン']
	 で 助詞 接続助詞 * * ctype * ['*', 'で', 'デ']
	 い 動詞 非自立 * * ctype 一段 ['連用形', 'いる', 'イ']
	 ます 助動詞 * * * ctype 特殊

In [22]:
run("マリアさんは中国しゅっしんです。")

response:
マリア 0
さん 3
は 5
中国 6
しゅっし 8
ん 12
です 13
。 15
0
	 マリア 名詞 固有名詞 一般 * ctype * ['*', 'マリア', 'マリア']
	 1 さん person
	 は 助詞 係助詞 * * ctype * ['*', 'は', 'ハ']
1
	 3 中国 location
	 しゅっし 名詞 サ変接続 * * ctype * ['*', 'しゅっし', 'シュッシ']
	 ん 名詞 非自立 一般 * ctype * ['*', 'ん', 'ン']
	 です 助動詞 * * * ctype 特殊・デス ['基本形', 'です', 'デス']
	 。 記号 句点 * * ctype * ['*', '。', '。']
